In [1]:
from azureml.core import Workspace, Experiment
Workspace.from_config()
ws = Workspace.from_config() #Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-123895
Azure region: southcentralus
Subscription id: a7e82b92-bbed-4497-a8df-009b71b94f9c
Resource group: aml-quickstarts-123895


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = 'STANDARD_D2_V2'
compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
compute_target = ComputeTarget.create(ws, 'compute-cluster', compute_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(0.5, 2.0),"--max_iter": choice(10, 20, 25, 50)}) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_amount = 0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training',
              entry_script='train.py',
              compute_target=compute_target)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=15,
                                     max_concurrent_runs=5
                                     ) ### YOUR CODE HERE ###

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_hyperdrive_model = hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(value=best_hyperdrive_model, filename='best_model.pkl')
best_hyperdrive_model

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path)
ds.to_pandas_dataframe().head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [7]:
from train import clean_data

#Use the clean_data function to clean your data.
x, y = clean_data(ds) #### YOUR DATA OBJECT HERE ###

x['y'] = y

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name='y',
    n_cross_validations=5)

In [9]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_1f22aec6-cdd5-4c9b-af27-a5fe1597ed66

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [10]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###        
best_automl_run, fitted_model = remote_run.get_output()
joblib.dump(value=fitted_model, filename='best_automl_model.pkl')


['best_automl_model.pkl']

In [11]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_1f22aec6-cdd5-4c9b-af27-a5fe1597ed66_36,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  l1_ratio=0.836734693877551,
                                                                                                  learning_rate='constant',
                                                                                                  loss='modified_huber',
                     

In [14]:
AmlCompute.delete(compute_target)

Current provisioning state of AmlCompute is "Deleting"

